In [ ]:
%pip install langchain
%pip install langchain-deepseek
%pip install langchain-ollama
%pip install python-dotenv

In [5]:

from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env",override=True)
from os import getenv

In [ ]:
from langchain_deepseek import ChatDeepSeek

key = getenv("DEEPSEEK_API_KEY") 
url = getenv("DEEPSEEK_API_BASE")
model = getenv("DEEPSEEK_MODEL")

# print(key, url, model)

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="deepseek-r1:latest",
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg)

In [ ]:
import os

key = getenv("SF_API_KEY") 
url = getenv("SF_API_BASE")
model = getenv("SF_MODEL")

os.environ["DEEPSEEK_API_KEY"] = key
os.environ["DEEPSEEK_API_BASE"] = url
print(key, url, model)

In [ ]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to chinese. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_deepseek import ChatDeepSeek


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)
messages = [
    SystemMessage("Translate the following from English into Chinese."),
    HumanMessage("hi!"),
]

print(llm.invoke(messages).content)

In [7]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

system_template = "Translate the following from English into {language}"
human_template = "{text}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), 
     ("user", human_template)]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))


你好！


In [9]:
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

system_template = SystemMessagePromptTemplate.from_template( "Translate the following from English into {language}")
human_template = HumanMessagePromptTemplate.from_template("{text}")

prompt_template = ChatPromptTemplate.from_messages(
    [system_template, 
     human_template]
)

chain = prompt_template | llm | StrOutputParser()
# print(chain)
print(chain.invoke({"language": "Chinese", "text": "hi!"}))

你好！


In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)
input_data = "What is the capital of France?"
summary_chain = (
    ChatPromptTemplate.from_template("Summarize this: {input}")
    | llm
    | StrOutputParser()
)
sentiment_chain = (
    ChatPromptTemplate.from_template("What is the sentiment of this: {input}")
    | llm
    | StrOutputParser()
)

parallel_chain = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})
parallel_result = parallel_chain.invoke({"input": input_data})
print(parallel_result)

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch


llm = ChatDeepSeek(
    model=model,
    temperature=0,
    max_retries=2,
    # other params...
)

# 定义子链
tech_prompt = ChatPromptTemplate.from_template(
    "作为科技作者，用专业术语解释：{concept}"
)
sports_prompt = ChatPromptTemplate.from_template(
    "作为体育解说员，用生动语言描述：{concept}"
)
general_prompt = ChatPromptTemplate.from_template(
    "用通俗语言解释：{concept}"
)

tech_chain = tech_prompt | llm | StrOutputParser()
sports_chain = sports_prompt | llm | StrOutputParser()
general_chain = general_prompt | llm | StrOutputParser()

# 构建分支逻辑
branch_chain = RunnableBranch(
    (lambda x: x["topic"] == "科技", tech_chain),
    (lambda x: x["topic"] == "体育", sports_chain),
    general_chain
)

# 完整执行链
# full_chain = RunnableParallel({  # 并行处理输入
#     "concept": lambda x: x["concept"],
#     "topic": lambda x: x["topic"]
# }) | branch_chain
full_chain = branch_chain

# 测试不同分支
print(full_chain.invoke({
    "concept": "神经网络", 
    "topic": "科技"
}))
# 输出包含"神经元"、"反向传播"等术语

print(full_chain.invoke({
    "concept": "越位", 
    "topic": "体育"
}))
# 输出包含"足球比赛"、"裁判判罚"等描述
